In [1]:
import sqlalchemy as db
import pandas as pd
import os
import random

In [2]:
class game_assets():
    def load_game_db(self):
        DBUSER= os.getenv('DBUSER','postgres')
        DBPASSWORD = os.getenv('DBPASSWORD','shadow954')
        DBHOST = os.getenv('DBHOST','localhost')
        DBPORT = os.getenv('DBPORT','5432')
        DBNAME = os.getenv('DBNAME','rust-items')

        engine = db.create_engine(
            f"postgresql+psycopg2://{DBUSER}:{DBPASSWORD}@{DBHOST}:{DBPORT}/{DBNAME}", client_encoding='utf8')
        return engine
    
    def start_game(self):
        print('Seja bem-vindo!')
        
        conta_ex = 0 
        while(conta_ex!='S' or conta_ex!='N'):
            print('Já possui uma conta? S ou N')
            conta_ex = input()
            if(conta_ex == 'S'):
                print('ótimo, qual o id da sua conta?')
                maincharacterlist = pd.read_sql('select * from maincharacter',engine)
                print(maincharacterlist)
                idConta = input()
                return idConta
            elif(conta_ex == 'N'):
                print('deseja criar uma? S ou N')
                createAcc = input()
                if(createAcc == 'S'):
                    print('qual será o nome da conta?')
                    owner = input()
                    print('Ótimo! Sua conta foi criada com o nome {}'.format(owner))
                    insertion = "INSERT INTO maincharacter(owner) VALUES('{}')".format(owner)
                    with engine.begin() as conn:     # TRANSACTION
                        conn.execute(insertion)
                    return pd.read_sql('select id from maincharacter order by id desc limit 1',engine).id
                elif(createAcc== 'N'):
                    print('Conta não criada')     

        return owner
    
    def instance_loot_box(self,loot_grade):
        query = "select * from lootcrates where grade = '{}'".format(loot_grade)
        qoi = pd.read_sql(query,engine).item_quantity
        
        iqoi = random.randint(2,int(qoi))
        
        query = """SELECT id FROM items WHERE lootgrade = '{}' ORDER BY random() LIMIT {};  """.format(loot_grade,iqoi)
        
        lootbox_items = pd.read_sql(query, engine)
        
        columns = pd.read_sql('select * from loot_crate_instance',engine).columns
        
        if iqoi == 2:
            insertion = "INSERT INTO loot_crate_instance(loot_grade,item1_id,item2_id) VALUES ('{}',{},{})".format(loot_grade, lootbox_items.id[0],lootbox_items.id[1])
        elif iqoi == 3:
            insertion = "INSERT INTO loot_crate_instance(loot_grade,item1_id,item2_id,item3_id) VALUES ('{}',{},{},{})".format(loot_grade, lootbox_items.id[0],lootbox_items.id[1],lootbox_items.id[2])
        elif iqoi == 4:
            insertion = "INSERT INTO loot_crate_instance(loot_grade,item1_id,item2_id,item3_id,item4_id) VALUES ('{}',{},{},{},{})".format(loot_grade, lootbox_items.id[0],lootbox_items.id[1],lootbox_items.id[2],lootbox_items.id[3])
        elif iqoi == 5:
            insertion = "INSERT INTO loot_crate_instance(loot_grade,item1_id,item2_id,item3_id,item4_id,item5_id) VALUES ('{}',{},{},{},{},{})".format(loot_grade, lootbox_items.id[0],lootbox_items.id[1],lootbox_items.id[2],lootbox_items.id[3],lootbox_items.id[4])
        elif iqoi == 6:
            insertion = "INSERT INTO loot_crate_instance(loot_grade,item1_id,item2_id,item3_id,item4_id,item5_id,item6_id) VALUES ('{}',{},{},{},{},{},{})".format(loot_grade, lootbox_items.id[0],lootbox_items.id[1],lootbox_items.id[2],lootbox_items.id[3],lootbox_items.id[4],lootbox_items.id[5])
        elif iqoi == 7:
            insertion = "INSERT INTO loot_crate_instance(loot_grade,item1_id,item2_id,item3_id,item4_id,item5_id,item6_id,item7_id) VALUES ('{}',{},{},{},{},{},{},{})".format(loot_grade, lootbox_items.id[0],lootbox_items.id[1],lootbox_items.id[2],lootbox_items.id[3],lootbox_items.id[4],lootbox_items.id[5],lootbox_items.id[6])        
                             
        with engine.begin() as conn:     # TRANSACTION
            conn.execute(insertion)
            
        return int(pd.read_sql("select id from loot_crate_instance order by id desc limit 1;",engine).id)
    
#       def instanciate_backpack(self, ownerID):
#         query = "INSERT INTO backpack(ownerID, slot01,slot02) VALUES({},) ".format(ownerID)    
#         with engine.begin() as conn:     # TRANSACTION
#             conn.execute(drop_query)

    def quit_game(self):
        drop_query = 'DELETE FROM loot_crate_instance;'
        with engine.begin() as conn:     # TRANSACTION
            conn.execute(drop_query)

In [11]:
pd.read_sql('select * from ITEMS',engine)

,id,maxstacksize,itemname,itemtype,lootgrade
0,36,1,Machete,melee,tool
1,37,1,Salvage Sword,melee,basic
2,38,1,Salvage Cleaver,melee,basic
3,39,1,Longsword,melee,military
4,40,1,Mace,melee,basic
...,...,...,...,...,...
118,119,5,Advanced Radiation Tea,tea,food
119,120,5,Pure Radiation Tea,tea,food
120,121,5,Normal Radiation Resistance Tea,tea,food
121,122,5,Advanced Radiation Resistance Tea,tea,food


In [ ]:
with engine.begin() as conn:     # TRANSACTION
    conn.execute('DELETE FROM maincharacter loot_crate_instance')

In [4]:
gi = game_assets()
engine = gi.load_game_db()

In [5]:
instanceid = gi.instance_loot_box('elite')

In [7]:
gi.start_game()

Seja bem-vindo!
Já possui uma conta? S ou N
N
deseja criar uma? S ou N
S
qual será o nome da conta?
baiano
Ótimo! Sua conta foi criada com o nome baiano


IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "maincharacter" violates foreign key constraint "fk_maincharacter_characters"
DETAIL:  Key (charactersid)=(6) is not present in table "characters".

[SQL: INSERT INTO maincharacter(owner) VALUES('baiano')]
(Background on this error at: http://sqlalche.me/e/14/gkpj)

In [ ]:
gi.quit_game()

In [ ]:
pd.read_sql('select * from maincharacter',engine)

In [ ]:
pd.read_sql('select * from lootbox',engine).id[0]

In [ ]:
pd.read_sql('select * from items where id=58',engine)

In [ ]:
pd.read_sql('select * from lootcrates',engine)